In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split

# Load the data
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')

# Preprocess the data
for data in [train_data, test_data]:
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['day'] = data['date'].dt.day

drop_columns = ['id', 'site_id', 'date', 'city', 'country']

In [8]:
for data in [train_data, test_data]:
    for col in data.select_dtypes(include=np.number).columns:
        if data[col].isnull().any():
            data[col].fillna(data[col].median(), inplace=True)

In [9]:
# Separate target variable and features
X_train = train_data.drop(columns=drop_columns + ['pm2_5'])
y_train = train_data['pm2_5']

X_test = test_data.drop(columns=drop_columns)
ids_test = test_data['id']

In [10]:
X_train.head()

,site_latitude,site_longitude,hour,sulphurdioxide_so2_column_number_density,sulphurdioxide_so2_column_number_density_amf,sulphurdioxide_so2_slant_column_number_density,sulphurdioxide_cloud_fraction,sulphurdioxide_sensor_azimuth_angle,sulphurdioxide_sensor_zenith_angle,sulphurdioxide_solar_azimuth_angle,...,cloud_cloud_base_pressure,cloud_cloud_base_height,cloud_cloud_optical_depth,cloud_surface_albedo,cloud_sensor_azimuth_angle,cloud_sensor_zenith_angle,cloud_solar_azimuth_angle,cloud_solar_zenith_angle,year,day
0,6.53257,3.39936,13,0.000012,0.669632,0.000007,0.123876,71.653160,42.564364,-95.848477,...,60432.792969,4389.787844,8.752905,0.257323,-97.477511,49.187592,-74.597511,29.002745,2023,25
1,6.53257,3.39936,12,0.000012,0.669632,0.000007,0.123876,71.653160,42.564364,-95.848477,...,60432.792969,4389.787844,8.752905,0.257323,-97.477511,49.187592,-74.597511,29.002745,2023,2
2,6.53257,3.39936,13,0.000012,0.669632,0.000007,0.123876,71.653160,42.564364,-95.848477,...,51171.802486,5791.682829,11.816715,0.192757,-96.411890,61.045123,-121.307414,41.898269,2023,3
3,6.53257,3.39936,14,0.000012,0.669632,0.000007,0.123876,71.653160,42.564364,-95.848477,...,60432.792969,4389.787844,8.752905,0.257323,-97.477511,49.187592,-74.597511,29.002745,2023,8
4,6.53257,3.39936,13,0.000267,0.774656,0.000207,0.223403,-97.811241,49.513344,-126.064468,...,96215.906250,451.050598,10.521009,0.153114,-97.811241,49.513439,-126.064453,40.167355,2023,9


In [11]:
# Scale the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# Reshape the input data for CNN
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_val_scaled = X_val_scaled.reshape(X_val_scaled.shape[0], X_val_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [13]:
# Build the CNN model
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

In [14]:
model.compile(optimizer='adam', loss='mse')

In [15]:
epochs = 50
batch_size = 32
model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/50
202/202 [==============================] - 2s 6ms/step - loss: 638.1871 - val_loss: 643.8115
Epoch 2/50
202/202 [==============================] - 1s 5ms/step - loss: 572.3422 - val_loss: 602.8655
Epoch 3/50
202/202 [==============================] - 1s 5ms/step - loss: 548.4980 - val_loss: 606.3687
Epoch 4/50
202/202 [==============================] - 1s 6ms/step - loss: 532.9992 - val_loss: 620.2573
Epoch 5/50
202/202 [==============================] - 1s 5ms/step - loss: 520.6312 - val_loss: 618.4549
Epoch 6/50
202/202 [==============================] - 1s 5ms/step - loss: 511.0084 - val_loss: 641.6024
Epoch 7/50
202/202 [==============================] - 1s 5ms/step - loss: 509.8543 - val_loss: 601.9093
Epoch 8/50
202/202 [==============================] - 1s 5ms/step - loss: 494.1701 - val_loss: 606.8414
Epoch 9/50
202/202 [==============================] - 1s 6ms/step - loss: 488.6956 - val_loss: 610.6554
Epoch 10/50
202/202 [==============================] - 1s 6ms/st

In [16]:
predictions = model.predict(X_test_scaled).flatten()

# Create a DataFrame for the predictions
predictions_df = pd.DataFrame({
    'id': ids_test,
    'pm2_5': predictions
})

# Save the predictions to a CSV file
predictions_df.to_csv('test_predictions_cnn.csv', index=False)

87/87 [==============================] - 0s 2ms/step
